# Quantitative Commodity Pricing Engine
## Complete Implementation: WTI Crude Oil Asian Options

**Author:** Abhishek  
**Date:** December 2025  
**Version:** 1.0

---

## Overview

This notebook implements a complete quantitative pricing engine for commodity derivatives:

- **Module 1:** Data Ingestion & Parameter Estimation
- **Module 2:** Monte Carlo Simulation (Ornstein-Uhlenbeck Process)
- **Module 3:** Asian Call Option Pricing
- **Module 4:** Risk Analysis & Visualization

---

### Key Features:
- ✅ Mean-reverting stochastic process (OU) for commodities
- ✅ 10,000 Monte Carlo simulations
- ✅ Asian option pricing (arithmetic average)
- ✅ Greeks calculation (Delta)
- ✅ Professional visualizations

---
## Cell 1: Imports and Setup
Run this cell first to import all required libraries.

In [ ]:
# Install yfinance if needed (run once)
# !pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Tuple, Optional, Dict
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for Jupyter
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

print("✅ All libraries imported successfully!")
print(f"📊 NumPy version: {np.__version__}")
print(f"📊 Pandas version: {pd.__version__}")
print(f"📊 Matplotlib version: {plt.matplotlib.__version__}")

---
## Cell 2: Module 1 - Data Ingestion & Parameter Estimation

This module downloads historical commodity data and calculates key parameters:
- Current spot price (S₀)
- Long-term mean (μ)
- Annualized volatility (σ)

In [ ]:
class CommodityData:
    """
    A class to handle commodity futures data download, processing, and analysis.
    """
    
    def __init__(self, ticker: str, years: int = 5):
        self.ticker = ticker
        self.years = years
        self.data = None
        self.log_returns = None
        self.annualized_volatility = None
        self.mean_reversion_level = None
        
    def download_data(self, use_simulation: bool = False) -> pd.DataFrame:
        """Download historical data from Yahoo Finance, or use simulated data."""
        if use_simulation:
            print(f"📈 Generating simulated {self.years}-year data for {self.ticker}...")
            self.data = self._generate_simulated_data()
            print(f"✅ Successfully generated {len(self.data)} data points\n")
            return self.data
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365 * self.years)
        
        print(f"🔄 Downloading {self.years}-year historical data for {self.ticker}...")
        print(f"📅 Period: {start_date.date()} to {end_date.date()}")
        
        try:
            self.data = yf.download(
                self.ticker,
                start=start_date,
                end=end_date,
                progress=False
            )
            
            if self.data.empty:
                raise ValueError(f"No data downloaded for {self.ticker}")
            
            print(f"✅ Successfully downloaded {len(self.data)} data points\n")
            return self.data
        except Exception as e:
            print(f"⚠️ Download failed: {str(e)[:100]}")
            print("🔄 Falling back to simulated data...\n")
            return self.download_data(use_simulation=True)
    
    def _generate_simulated_data(self) -> pd.DataFrame:
        """Generate realistic simulated WTI Crude Oil price data."""
        S0 = 70.0
        mu = 0.0001
        sigma = 0.025
        theta = 0.15
        mean_price = 75.0
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365 * self.years)
        dates = pd.date_range(start=start_date, end=end_date, freq='B')
        
        n_days = len(dates)
        np.random.seed(42)
        prices = np.zeros(n_days)
        prices[0] = S0
        
        for t in range(1, n_days):
            drift = theta * (mean_price - prices[t-1]) / prices[t-1]
            shock = sigma * np.random.randn()
            prices[t] = prices[t-1] * np.exp(drift + shock)
        
        df = pd.DataFrame({
            'Open': prices * (1 + np.random.randn(n_days) * 0.002),
            'High': prices * (1 + np.abs(np.random.randn(n_days)) * 0.01),
            'Low': prices * (1 - np.abs(np.random.randn(n_days)) * 0.01),
            'Close': prices,
            'Volume': np.random.randint(100000, 500000, n_days)
        }, index=dates)
        
        df['High'] = df[['Open', 'High', 'Close']].max(axis=1)
        df['Low'] = df[['Open', 'Low', 'Close']].min(axis=1)
        
        return df
    
    def calculate_log_returns(self) -> pd.Series:
        """Calculate daily log returns from prices."""
        if self.data is None:
            raise ValueError("Data not loaded. Call download_data() first.")
        
        prices = self.data['Close']
        self.log_returns = np.log(prices / prices.shift(1))
        self.log_returns = self.log_returns.dropna()
        
        return self.log_returns
    
    def calculate_annualized_volatility(self, trading_days: int = 252) -> float:
        """Calculate annualized volatility from daily log returns."""
        if self.log_returns is None:
            raise ValueError("Log returns not calculated.")
        
        daily_volatility = self.log_returns.std()
        self.annualized_volatility = daily_volatility * np.sqrt(trading_days)
        
        return self.annualized_volatility
    
    def calculate_mean_reversion_level(self) -> float:
        """Calculate the long-term historical mean price."""
        if self.data is None:
            raise ValueError("Data not loaded.")
        
        self.mean_reversion_level = self.data['Close'].mean()
        return self.mean_reversion_level
    
    def analyze(self, use_simulation: bool = False) -> dict:
        """Perform complete analysis."""
        self.download_data(use_simulation=use_simulation)
        self.calculate_log_returns()
        self.calculate_annualized_volatility()
        self.calculate_mean_reversion_level()
        
        results = {
            'ticker': self.ticker,
            'data_points': len(self.data),
            'start_date': self.data.index[0],
            'end_date': self.data.index[-1],
            'current_price': self.data['Close'].iloc[-1],
            'mean_reversion_level': self.mean_reversion_level,
            'annualized_volatility': self.annualized_volatility,
            'volatility_percentage': self.annualized_volatility * 100,
            'min_price': self.data['Close'].min(),
            'max_price': self.data['Close'].max(),
        }
        
        return results
    
    def print_summary(self):
        """Print a formatted summary."""
        if self.data is None:
            raise ValueError("Analysis not performed.")
        
        print("=" * 70)
        print("📊 MODULE 1: PARAMETER ESTIMATION RESULTS")
        print("=" * 70)
        print(f"\n🛢️  Commodity Ticker: {self.ticker}")
        print(f"📅 Analysis Period: {self.data.index[0].date()} to {self.data.index[-1].date()}")
        print(f"📈 Total Data Points: {len(self.data)}")
        print(f"\n{'PRICE STATISTICS':^70}")
        print("-" * 70)
        print(f"Current Price (S₀):         ${self.data['Close'].iloc[-1]:.2f}")
        print(f"Mean Reversion Level (μ):   ${self.mean_reversion_level:.2f}")
        print(f"Historical Minimum:         ${self.data['Close'].min():.2f}")
        print(f"Historical Maximum:         ${self.data['Close'].max():.2f}")
        print(f"\n{'VOLATILITY METRICS':^70}")
        print("-" * 70)
        print(f"Annualized Volatility (σ):  {self.annualized_volatility:.4f} ({self.annualized_volatility * 100:.2f}%)")
        print(f"Daily Volatility:           {self.log_returns.std():.4f}")
        print("=" * 70)
        print()

print("✅ CommodityData class loaded successfully!")

---
## Cell 3: Module 2 - Monte Carlo Simulation Engine

Implements the **Ornstein-Uhlenbeck (OU) process** for mean-reverting commodities:

$$dX_t = \theta(\mu - X_t)dt + \sigma dW_t$$

Where:
- $\theta$ = Mean reversion speed
- $\mu$ = Long-term mean
- $\sigma$ = Volatility
- $dW_t$ = Brownian motion

In [ ]:
class MonteCarloEngine:
    """
    Monte Carlo simulation engine using Ornstein-Uhlenbeck process.
    
    Perfect for commodities because prices revert to long-term equilibrium.
    """
    
    def __init__(self, random_seed: Optional[int] = None):
        self.random_seed = random_seed
        self.simulated_paths = None
        self.time_grid = None
        
    def simulate_ou_process(
        self,
        S0: float,
        mu: float,
        theta: float,
        sigma: float,
        T: float,
        n_steps: int,
        n_simulations: int
    ) -> np.ndarray:
        """
        Simulate commodity price paths using Ornstein-Uhlenbeck process.
        
        MATHEMATICAL FOUNDATION:
        ========================
        The OU SDE: dX_t = θ(μ - X_t)dt + σ dW_t
        
        Components:
        - θ(μ - X_t)dt: DRIFT (mean reversion)
          * X_t > μ → drift negative → price pulled DOWN
          * X_t < μ → drift positive → price pulled UP
          
        - σ dW_t: DIFFUSION (random shocks)
          * Supply/demand, geopolitics, weather
        
        DISCRETE FORM (Euler-Maruyama):
        X_{t+Δt} = X_t + θ(μ - X_t)Δt + σ√(Δt)Z
        
        where Z ~ N(0,1)
        """
        if self.random_seed is not None:
            np.random.seed(self.random_seed)
        
        # Time discretization
        dt = T / n_steps
        sqrt_dt = np.sqrt(dt)
        
        self.time_grid = np.linspace(0, T, n_steps + 1)
        
        # Initialize price matrix
        prices = np.zeros((n_simulations, n_steps + 1))
        prices[:, 0] = S0
        
        # Simulate each time step
        for t in range(n_steps):
            Z = np.random.randn(n_simulations)
            
            current_prices = prices[:, t]
            drift = theta * (mu - current_prices) * dt
            diffusion = sigma * sqrt_dt * Z
            
            prices[:, t + 1] = current_prices + drift + diffusion
        
        self.simulated_paths = prices
        return prices
    
    def get_terminal_prices(self) -> np.ndarray:
        """Get final prices at maturity."""
        if self.simulated_paths is None:
            raise ValueError("No simulations run yet.")
        return self.simulated_paths[:, -1]
    
    def print_simulation_summary(self):
        """Print summary statistics."""
        if self.simulated_paths is None:
            raise ValueError("No simulations run yet.")
        
        terminal_prices = self.get_terminal_prices()
        
        print("=" * 70)
        print("🎲 MODULE 2: MONTE CARLO SIMULATION RESULTS")
        print("=" * 70)
        print(f"\n⚙️  Simulation Parameters:")
        print(f"  Number of Paths:     {self.simulated_paths.shape[0]:,}")
        print(f"  Time Steps:          {self.simulated_paths.shape[1] - 1:,}")
        print(f"  Initial Price:       ${self.simulated_paths[0, 0]:.2f}")
        print(f"\n📊 Terminal Price Statistics (at Maturity):")
        print(f"  Mean:                ${terminal_prices.mean():.2f}")
        print(f"  Median:              ${np.median(terminal_prices):.2f}")
        print(f"  Std Dev:             ${terminal_prices.std():.2f}")
        print(f"  Min:                 ${terminal_prices.min():.2f}")
        print(f"  Max:                 ${terminal_prices.max():.2f}")
        print(f"  5th Percentile:      ${np.percentile(terminal_prices, 5):.2f}")
        print(f"  95th Percentile:     ${np.percentile(terminal_prices, 95):.2f}")
        print("=" * 70)
        print()

print("✅ MonteCarloEngine class loaded successfully!")

---
## Cell 4: Module 3 - Option Pricing

Prices **Asian Call Options** using Monte Carlo valuation.

### Asian Option Payoff:
$$\text{Payoff} = \max\left(\bar{S} - K, 0\right)$$

Where $\bar{S}$ = arithmetic average price over the option's life

### Why Asian Options?
- Perfect for airlines hedging fuel costs
- Less volatile than European options
- 20-30% cheaper premiums

In [ ]:
class OptionPricer:
    """
    Option pricing engine for Asian call options.
    """
    
    def __init__(self, monte_carlo_engine: MonteCarloEngine):
        self.mc_engine = monte_carlo_engine
        self.option_price = None
        self.payoffs = None
        
    def price_asian_call(
        self,
        strike: float,
        risk_free_rate: float,
        T: float
    ) -> float:
        """
        Price an Asian Call Option.
        
        PAYOFF CALCULATION:
        ===================
        For each path:
        1. A = average price over path
        2. Payoff = max(A - K, 0)
        
        PRESENT VALUE:
        ==============
        C = e^(-rT) × E[max(A - K, 0)]
        """
        if self.mc_engine.simulated_paths is None:
            raise ValueError("No simulations available.")
        
        paths = self.mc_engine.simulated_paths
        
        # Calculate arithmetic average for each path
        average_prices = np.mean(paths, axis=1)
        
        # Calculate payoff for each path
        self.payoffs = np.maximum(average_prices - strike, 0)
        
        # Expected payoff
        expected_payoff = np.mean(self.payoffs)
        
        # Discount to present value
        discount_factor = np.exp(-risk_free_rate * T)
        self.option_price = discount_factor * expected_payoff
        
        return self.option_price
    
    def calculate_delta(
        self,
        S0: float,
        mu: float,
        theta: float,
        sigma: float,
        T: float,
        n_steps: int,
        n_simulations: int,
        strike: float,
        risk_free_rate: float,
        bump_size: float = 1.0
    ) -> float:
        """
        Calculate Delta: sensitivity to spot price.
        
        Δ ≈ (C(S₀ + h) - C(S₀)) / h
        """
        price_at_S0 = self.option_price
        
        # Simulate at bumped spot
        mc_bumped = MonteCarloEngine(random_seed=self.mc_engine.random_seed)
        mc_bumped.simulate_ou_process(
            S0=S0 + bump_size,
            mu=mu,
            theta=theta,
            sigma=sigma,
            T=T,
            n_steps=n_steps,
            n_simulations=n_simulations
        )
        
        pricer_bumped = OptionPricer(mc_bumped)
        price_at_S0_plus_bump = pricer_bumped.price_asian_call(strike, risk_free_rate, T)
        
        delta = (price_at_S0_plus_bump - price_at_S0) / bump_size
        return delta
    
    def print_pricing_summary(self, strike: float, delta: Optional[float] = None):
        """Print pricing results."""
        if self.option_price is None:
            raise ValueError("Option not priced yet.")
        
        print("=" * 70)
        print("💰 MODULE 3: ASIAN CALL OPTION PRICING RESULTS")
        print("=" * 70)
        print(f"\n📋 Option Specification:")
        print(f"  Type:                Asian Call (Arithmetic Average)")
        print(f"  Strike Price (K):    ${strike:.2f}")
        print(f"\n💵 Pricing Results:")
        print(f"  OPTION PRICE:        ${self.option_price:.4f}")
        print(f"  Expected Payoff:     ${np.mean(self.payoffs):.4f}")
        print(f"  Payoff Std Dev:      ${np.std(self.payoffs):.4f}")
        print(f"  Max Payoff:          ${np.max(self.payoffs):.4f}")
        print(f"  % In-The-Money:      {100 * np.sum(self.payoffs > 0) / len(self.payoffs):.2f}%")
        
        if delta is not None:
            print(f"\n📐 Greeks:")
            print(f"  Delta (Δ):           {delta:.4f}")
            print(f"  Interpretation:      ${1:.2f} spot ↑ → ${delta:.4f} option ↑")
        
        print("=" * 70)
        print()

print("✅ OptionPricer class loaded successfully!")

---
## Cell 5: Module 4 - Risk Analysis & Visualization

Creates professional visualizations:
- Monte Carlo price paths (showing mean reversion)
- Option payoff distribution

In [ ]:
class RiskAnalyzer:
    """Risk analysis and visualization toolkit."""
    
    def __init__(
        self,
        commodity_data: CommodityData,
        mc_engine: MonteCarloEngine,
        option_pricer: OptionPricer
    ):
        self.commodity_data = commodity_data
        self.mc_engine = mc_engine
        self.option_pricer = option_pricer
    
    def plot_price_paths(self, n_paths: int = 50, mu: Optional[float] = None):
        """Visualize simulated price paths."""
        if self.mc_engine.simulated_paths is None:
            raise ValueError("No simulations available.")
        
        paths = self.mc_engine.simulated_paths
        time_grid = self.mc_engine.time_grid
        
        if mu is None:
            mu = self.commodity_data.mean_reversion_level
        
        n_total = paths.shape[0]
        indices = np.random.choice(n_total, size=min(n_paths, n_total), replace=False)
        selected_paths = paths[indices, :]
        
        fig, ax = plt.subplots(figsize=(14, 6))
        
        for i in range(selected_paths.shape[0]):
            ax.plot(time_grid, selected_paths[i, :], alpha=0.3, linewidth=1, color='steelblue')
        
        ax.axhline(y=mu, color='red', linestyle='--', linewidth=2.5,
                  label=f'Long-Term Mean (μ = ${mu:.2f})', zorder=10)
        
        S0 = paths[0, 0]
        ax.axhline(y=S0, color='green', linestyle=':', linewidth=2,
                  label=f'Initial Price (S₀ = ${S0:.2f})', zorder=10)
        
        mean_path = np.mean(paths, axis=0)
        ax.plot(time_grid, mean_path, color='darkblue', linewidth=3,
               label='Mean Path', zorder=11)
        
        ax.set_xlabel('Time (Years)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Price (USD)', fontsize=12, fontweight='bold')
        ax.set_title(
            f'Monte Carlo Simulation: {selected_paths.shape[0]} Price Paths\n'
            f'Ornstein-Uhlenbeck Mean-Reverting Process',
            fontsize=14, fontweight='bold'
        )
        ax.legend(loc='best', fontsize=10, framealpha=0.9)
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        return fig
    
    def plot_payoff_distribution(self, strike: float):
        """Visualize option payoff distribution."""
        if self.option_pricer.payoffs is None:
            raise ValueError("No payoffs calculated.")
        
        payoffs = self.option_pricer.payoffs
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        # All payoffs
        ax1.hist(payoffs, bins=50, color='steelblue', alpha=0.7,
                edgecolor='black', linewidth=0.5)
        ax1.axvline(x=np.mean(payoffs), color='red', linestyle='--', linewidth=2,
                   label=f'Mean: ${np.mean(payoffs):.4f}')
        ax1.set_xlabel('Payoff (USD)', fontsize=11, fontweight='bold')
        ax1.set_ylabel('Frequency', fontsize=11, fontweight='bold')
        ax1.set_title('Distribution of Option Payoffs', fontsize=12, fontweight='bold')
        ax1.legend(loc='best', fontsize=9)
        ax1.grid(True, alpha=0.3)
        
        # In-the-money only
        itm_payoffs = payoffs[payoffs > 0]
        ax2.hist(itm_payoffs, bins=30, color='green', alpha=0.7,
                edgecolor='black', linewidth=0.5)
        if len(itm_payoffs) > 0:
            ax2.axvline(x=np.mean(itm_payoffs), color='darkgreen',
                       linestyle='--', linewidth=2,
                       label=f'Mean ITM: ${np.mean(itm_payoffs):.4f}')
        ax2.set_xlabel('Payoff (USD)', fontsize=11, fontweight='bold')
        ax2.set_ylabel('Frequency', fontsize=11, fontweight='bold')
        ax2.set_title(
            f'In-The-Money Payoffs Only\n'
            f'({100 * len(itm_payoffs) / len(payoffs):.1f}% of simulations)',
            fontsize=12, fontweight='bold'
        )
        ax2.legend(loc='best', fontsize=9)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        return fig

print("✅ RiskAnalyzer class loaded successfully!")
print("\n" + "="*70)
print("✅ ALL CLASSES LOADED SUCCESSFULLY!")
print("="*70)
print("\n📝 You can now proceed to the next cell to run the analysis!")

---
## Cell 6: Execute Complete Analysis

This cell runs the entire pricing engine workflow:
1. Download commodity data
2. Run Monte Carlo simulation
3. Price Asian call option
4. Calculate Delta
5. Generate visualizations

### ⚙️ Configuration:
- **Commodity:** WTI Crude Oil (`CL=F`)
- **Simulations:** 10,000 paths
- **Time Steps:** 252 (daily)
- **Time Horizon:** 1 year
- **Strike:** At-the-money (current spot)

### 🔧 Important Settings:
**If behind a firewall**, change `use_simulation=False` to `use_simulation=True`

In [ ]:
print("\n" + "🚀"*35)
print("STARTING QUANTITATIVE COMMODITY PRICING ENGINE")
print("🚀"*35 + "\n")

# ========================================================================
# STEP 1: DATA INGESTION & PARAMETER ESTIMATION
# ========================================================================
print("📊 STEP 1: Downloading and analyzing historical commodity data...")
print("-" * 70)

commodity = CommodityData(ticker='CL=F', years=5)

# IMPORTANT: If you're on a restricted network (like at work), 
# change use_simulation=False to use_simulation=True below
params = commodity.analyze(use_simulation=False)
commodity.print_summary()

# Extract parameters for simulation
S0 = params['current_price']
mu = params['mean_reversion_level']
sigma = params['annualized_volatility']

# ========================================================================
# STEP 2: MONTE CARLO SIMULATION (ORNSTEIN-UHLENBECK)
# ========================================================================
print("\n🎲 STEP 2: Running Monte Carlo simulation with OU process...")
print("-" * 70)

# Simulation parameters
theta = 0.15          # Mean reversion speed (calibrated for commodities)
T = 1.0              # 1 year time horizon
n_steps = 252        # Daily time steps (252 trading days)
n_simulations = 10000  # 10,000 price paths

print(f"\n⚙️  Simulation Configuration:")
print(f"  Mean Reversion Speed (θ): {theta}")
print(f"  Time Horizon (T):         {T} year")
print(f"  Time Steps:               {n_steps} (daily)")
print(f"  Number of Paths:          {n_simulations:,}")
print()

# Initialize and run simulation
mc_engine = MonteCarloEngine(random_seed=42)
simulated_paths = mc_engine.simulate_ou_process(
    S0=S0,
    mu=mu,
    theta=theta,
    sigma=sigma,
    T=T,
    n_steps=n_steps,
    n_simulations=n_simulations
)

mc_engine.print_simulation_summary()

# ========================================================================
# STEP 3: OPTION PRICING (ASIAN CALL)
# ========================================================================
print("\n💰 STEP 3: Pricing Asian Call Option...")
print("-" * 70)

# Option parameters
strike = S0  # At-the-money (ATM) option
risk_free_rate = 0.042  # 4.2% annual risk-free rate

print(f"\n📋 Option Configuration:")
print(f"  Strike Price (K):         ${strike:.2f}")
print(f"  Risk-Free Rate (r):       {risk_free_rate*100:.1f}%")
print(f"  Moneyness:                ATM (At-The-Money)")
print()

# Price the option
pricer = OptionPricer(mc_engine)
option_price = pricer.price_asian_call(
    strike=strike,
    risk_free_rate=risk_free_rate,
    T=T
)

print(f"\n📐 Calculating Greeks (Delta)...")
print("-" * 70)

# Calculate Delta
delta = pricer.calculate_delta(
    S0=S0,
    mu=mu,
    theta=theta,
    sigma=sigma,
    T=T,
    n_steps=n_steps,
    n_simulations=n_simulations,
    strike=strike,
    risk_free_rate=risk_free_rate,
    bump_size=1.0
)

pricer.print_pricing_summary(strike=strike, delta=delta)

# ========================================================================
# STEP 4: RISK ANALYSIS & VISUALIZATION
# ========================================================================
print("\n📈 STEP 4: Generating risk analysis and visualizations...")
print("-" * 70)

# Initialize risk analyzer
risk_analyzer = RiskAnalyzer(commodity, mc_engine, pricer)

# Create visualizations
print("\n📊 Generating price path visualization...")
fig1 = risk_analyzer.plot_price_paths(n_paths=50, mu=mu)
plt.show()

print("\n📊 Generating payoff distribution visualization...")
fig2 = risk_analyzer.plot_payoff_distribution(strike=strike)
plt.show()

# ========================================================================
# FINAL SUMMARY
# ========================================================================
print("\n" + "="*70)
print("✅ ANALYSIS COMPLETE - EXECUTIVE SUMMARY")
print("="*70)
print(f"""
🛢️  Market Parameters:
  Commodity:                 WTI Crude Oil Futures (CL=F)
  Current Spot Price:        ${S0:.2f}
  Long-Term Mean:            ${mu:.2f}
  Annualized Volatility:     {sigma*100:.2f}%
  Mean Reversion Speed:      {theta:.2f}

📋 Option Contract:
  Type:                      Asian Call (Arithmetic Average)
  Strike Price:              ${strike:.2f} (ATM)
  Time to Maturity:          {T:.1f} year
  Risk-Free Rate:            {risk_free_rate*100:.1f}%

💰 Pricing Results:
  ╔═══════════════════════════════════════╗
  ║  OPTION PRICE:  ${option_price:.4f} per barrel  ║
  ╚═══════════════════════════════════════╝
  
  Expected Payoff:           ${np.mean(pricer.payoffs):.4f}
  Probability ITM:           {100*np.sum(pricer.payoffs>0)/len(pricer.payoffs):.2f}%
  Delta:                     {delta:.4f}

💼 Business Interpretation:
  For a quarterly contract of 100,000 barrels:
  ┌─────────────────────────────────────────┐
  │ Total Premium:    ${option_price * 100000:>14,.2f}   │
  │ Hedge Ratio:      {delta:>14.2%}           │
  │ Protection:       Avg price ≤ ${strike:.2f}/bbl │
  └─────────────────────────────────────────┘
""")

print("="*70)
print("✅ All calculations complete!")
print("📊 All visualizations displayed above")
print("="*70 + "\n")

# Store results for further analysis
results = {
    'commodity': commodity,
    'mc_engine': mc_engine,
    'pricer': pricer,
    'risk_analyzer': risk_analyzer,
    'params': params,
    'option_price': option_price,
    'delta': delta
}

print("💾 Results stored in 'results' dictionary")
print("   Access with: results['option_price'], results['delta'], etc.")

---
## Additional Analysis (Optional)

Use the cells below to experiment with different parameters or perform custom analysis.

In [ ]:
# Example: Price an out-of-the-money option
otm_strike = S0 * 1.10  # 10% above current price

pricer_otm = OptionPricer(mc_engine)
price_otm = pricer_otm.price_asian_call(otm_strike, risk_free_rate, T)

print(f"Out-of-The-Money Option (Strike = ${otm_strike:.2f}):")
print(f"  Price: ${price_otm:.4f}")
print(f"  Prob ITM: {100*np.sum(pricer_otm.payoffs>0)/len(pricer_otm.payoffs):.2f}%")

In [ ]:
# Example: Access individual results
print("Quick Results Access:")
print(f"  Option Price: ${results['option_price']:.4f}")
print(f"  Delta: {results['delta']:.4f}")
print(f"  Current Spot: ${results['params']['current_price']:.2f}")
print(f"  Volatility: {results['params']['annualized_volatility']*100:.2f}%")

---
## Summary & Next Steps

### What We Built:
✅ Complete quantitative pricing engine for commodity derivatives  
✅ Ornstein-Uhlenbeck mean-reverting process  
✅ Asian option pricing with Monte Carlo  
✅ Greeks calculation (Delta)  
✅ Professional risk visualizations  

### Key Results:
- **Option Price:** Typically $0.04-0.05 per barrel for ATM options
- **Delta:** Shows hedge ratio for risk management
- **Mean Reversion:** Clearly visible in price path charts

### Next Steps:
1. **Experiment** with different strikes (ITM, OTM)
2. **Try other commodities** (Natural Gas, Gold, Silver)
3. **Calculate more Greeks** (Gamma, Vega, Theta)
4. **Build trading strategies** using the pricing model

---

**Questions or Issues?**
- Check the `SETUP_GUIDE.md` for troubleshooting
- Review `COMPLETE_DOCUMENTATION.md` for technical details
- See `QUICK_REFERENCE.md` for parameter tuning

**Happy Pricing! 🚀📈**